In [1]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, AutoTokenizer, AutoModelForCausalLM
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [22]:
model = GPT2LMHeadModel.from_pretrained("gunwoo723/kogpt-trinity-poem-generator").to(device)
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5",
  bos_token='<s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [23]:
tokenizer.add_special_tokens({'additional_special_tokens': ["<yun>"]})
model.resize_token_embeddings(len(tokenizer))

Embedding(51201, 1920)

In [24]:
sentence = '<s>멈춰버린 시계 속 마지막으로 남은 바램\n'
inputs = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='pt').to(device)

with torch.no_grad():
    output = model.generate(
        input_ids=inputs,
        temperature=0.5, # 생성 다양성 조절
        min_length=32, # 생성되는 문장의 최소 길이
        max_length=256, # 생성되는 문장의 최대 길이
        repetition_penalty=1.2, # 반복을 줄이는 패널티
        do_sample=True, # 샘플링 기반 생성 활성화
        early_stopping=True, # EOS token을 만나면 조기 종료
        eos_token_id=tokenizer.eos_token_id
    )

generated_sequence = output[0].tolist()
text = tokenizer.decode(generated_sequence, skip_special_tokens=False)

In [25]:
text = text.replace("<yun> ", "\n").replace("<s> ", "").replace("</s>", "")

In [26]:
print(text)

멈춰버린 시계 속 마지막으로 남은 바램
다시 돌아 올 수 없는 그 먼 곳 까지

홀로 타향에서 애타게 그리워 하는 
잊혀 진 기억속으로 한 걸음 다가서는 이
바람에 실려 오는 향수 묻어난다

세월의 더께 묻어 둔 채 흘러간 추억들
아쉬운 눈길로 다시금 불러 보는 이름
이제 다시 볼 수 없음에 서러워라

잊힌 기억마저도 되살려 주는 시간들이
갈색 낙엽으로 쌓여만 가는 어느 가을날
한없이 깊어져간 가슴속 사연들을
낙엽따라 떠나가는 하얀 눈송이처럼 날려보낸다
